In [ ]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [ ]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView("home_sales_view")
result = spark.sql("SELECT * FROM home_sales_view")
result.show()


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

result = spark.sql("SELECT YEAR(Date) AS Year, ROUND(AVG(Price), 2) AS AveragePrice FROM home_sales_view WHERE Bedrooms = 4 GROUP BY Year ORDER BY Year")
result.show()


In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

result = spark.sql("SELECT YEAR(Date) AS Year, ROUND(AVG(Price), 2) AS AveragePrice FROM home_sales_view WHERE Bedrooms = 3 AND Bathrooms = 3 GROUP BY Year ORDER BY Year")
result.show()



In [ ]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

result = spark.sql("SELECT YEAR(Date) AS Year, ROUND(AVG(Price), 2) AS AveragePrice FROM home_sales_view WHERE Bedrooms = 3 AND Bathrooms = 3 AND Floors = 2 AND Size >= 2000 GROUP BY Year ORDER BY Year")
result.show()


In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.


result = spark.sql("SELECT ROUND(AVG(Price), 2) AS AveragePrice FROM home_sales_view WHERE Price >= 350000")
average_price = result.first()["AveragePrice"]

view_rating = when(average_price >= 500000, "High").when(average_price >= 400000, "Medium").otherwise("Low")


print("View Rating: ", view_rating)

# Calculate the run time
start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")


In [ ]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached("home_sales")
print("Is 'home_sales' table cached? ", is_cached)

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

result = spark.sql("SELECT View, AVG(Price) AS AveragePrice FROM home_sales WHERE Price >= 350000 GROUP BY View")

result.show()


print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

home_sales_df.write.partitionBy("date_built").parquet("path/to/output_directory")


In [ ]:
# 11. Read the formatted parquet data.

formatted_parquet_data = spark.read.parquet("path/to/parquet_directory")


In [ ]:
# 12. Create a temporary table for the parquet data.

formatted_parquet_data.createOrReplaceTempView("parquet_table")


In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 




#  Parquet DataFrame as a temporary table
formatted_parquet_data.createOrReplaceTempView("parquet_table")

# filter out the view ratings with average price >= $350,000
result = spark.sql("""
    SELECT AVG(price) AS average_price
    FROM parquet_table
    WHERE price >= 350000
""")
result.show()


start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(is_cached)

